In [1]:
import sys
sys.path.insert(1, '../..')

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../../data/processed/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2,tvt2_1,tvt2_2,tvt2_3
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,validation,training,validation,training
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,training,validation,training,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,training,validation,validation,training
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training,training,training,training
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,training,validation,training,training


In [3]:
bigram_data = pd.read_excel('../../data/processed/twitter15_ngram_distribution.xlsx', sheet_name='bigram')
bigram_data.columns = ["token", "unverified", "non-rumor", "true", "false"]
bigram_data.head()

,token,unverified,non-rumor,true,false
0,ca kkk,1,0,0,0
1,kkk grand,1,0,0,0
2,grand wizard,1,0,0,0
3,wizard endorses,1,0,0,0
4,endorses @hillaryclinton,1,0,0,0


In [4]:
bigram_vector_base = bigram_data['token'].tolist()
print(len(bigram_vector_base))
bigram_vector_base[10]

13842


'to trump'

In [5]:
label_rnr = []
for i, d in data.iterrows():
    if d['label'] == "non-rumor":
        label_rnr.append("non-rumours")
    else:
        label_rnr.append("rumours")
        
data['label_rnr'] = pd.Series(label_rnr)
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2,tvt2_1,tvt2_2,tvt2_3,label_rnr
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,validation,training,validation,training,rumours
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,training,validation,training,training,rumours
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,training,validation,validation,training,rumours
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training,training,training,training,rumours
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,training,validation,training,training,rumours


In [6]:
# labels_str = ['rumour', 'non-rumour']
label_type = "label2"
labels_str = data[label_type].unique().tolist()
labels_str

['non-rumours', 'true', 'unverified', 'false']

In [7]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d[label_type])
    labels.append(lab)
labels[:10]

[0, 1, 1, 0, 0, 0, 0, 2, 0, 0]

In [8]:
import string
import nltk
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(reduce_len=True)


def text2unigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    unigrams = texts
    
    return unigrams


def text2bigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    bigrams = nltk.bigrams(texts)
    bigrams = map(' '.join, bigrams)
    bigrams = [bgr for bgr in bigrams]
    
    return bigrams


def text2trigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    trigrams = nltk.trigrams(texts)
    trigrams = map(' '.join, trigrams)
    trigrams = [bgr for bgr in trigrams]
    
    return trigrams


def bigrams_vectors_generation(texts):
    bigram_vectors = []
    for text in texts:
        bigrams = text2bigrams(text)

        init_vec = [0.0 for _ in range(len(bigram_vector_base) + 1)]
        for bgr in bigrams:
            if bgr in bigram_vector_base:
                idx = bigram_vector_base.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        bigram_vectors.append(init_vec)
    
    return bigram_vectors


def custom_vectors_generation(texts, vector_terms):
    vectors = []
    for text in texts:
        bigrams = text2bigrams(text)
        trigrams = text2trigrams(text)

        init_vec = [0.0 for _ in range(len(vector_terms) + 1)]
        for bgr in bigrams:
            if bgr in vector_terms:
                idx = vector_terms.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        for tgr in trigrams:
            if tgr in vector_terms:
                idx = vector_terms.index(tgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        vectors.append(init_vec)
    
    return vectors

In [9]:
texts = data['tweet_text'].tolist()
# vectors = bigrams_vectors_generation(texts)

In [10]:
import nltk
from nltk.collocations import *

top_n = 2000
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder2 = BigramCollocationFinder.from_words([])
finder3 = TrigramCollocationFinder.from_words([])

# generating bigram and trigram
for text in texts:
    unigrams = text2unigrams(text)
    bigrams = text2bigrams(text)
    trigrams = text2trigrams(text)
    
    for ngrm in unigrams:
        if ngrm not in finder2.word_fd:
            finder2.word_fd[ngrm] = 0
        finder2.word_fd[ngrm] += 1
        finder2.N += 1
        
        if ngrm not in finder3.word_fd:
            finder3.word_fd[ngrm] = 0
        finder3.word_fd[ngrm] += 1
        finder3.N += 1
    
    for ngrm in bigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder2.ngram_fd:
            finder2.ngram_fd[term] = 0
            
        finder2.ngram_fd[term] += 1

    for ngrm in trigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder3.ngram_fd:
            finder3.ngram_fd[term] = 0
            
        finder3.ngram_fd[term] += 1
        
# only bigrams that appear 3+ times
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(3)

combined = []
for res in finder2.score_ngrams(bigram_measures.pmi):
    combined.append(res)
for res in finder3.score_ngrams(trigram_measures.pmi):
    combined.append(res)
combined = sorted(combined, key=lambda x: x[1], reverse=True)

In [11]:
print(len(finder2.ngram_fd), len(finder3.ngram_fd))
count = 0
for k, v in combined:
    print(f"{k} - {v}")
    count += 1
    if count >= 10:
        break

5500 2977
('AG', 'Eric', 'Holder') - 30.1391512738226
('33', '55', '77') - 29.309076275264914
('Hormone', 'MV', 'Concept') - 29.309076275264914
('Last-minute', 'lawsuit', 'filed') - 27.5017213532073
('AC', 'Milan', 'midfielder') - 27.309076275264914
('serious', 'depressive', 'episode') - 26.899685339127203
('http://t.co/j1ljDbazOw', 'UPDATES', 'http://t.co/PD7pVfU9pQ') - 26.817223178935233
('Jewish', 'pro-censorship', 'lobby') - 26.665220085490187
('Mean', 'Russias', 'Undergoing') - 26.64729817749292
('altitude', 'speed', 'chart') - 26.50172135320731


In [12]:
term_vector_base = [" ".join(c[0]) for c in combined[:top_n]]
vectors = custom_vectors_generation(texts, term_vector_base)

In [13]:
len(vectors[102])

2001

In [14]:
min_score = 100
max_score = 0
n_2gram = 0
n_3gram = 0
for k, v in combined[:top_n]:
    min_score = min(min_score, v)
    max_score = max(max_score, v)
    
    if len(k) == 2:
        n_2gram += 1
    
    if len(k) == 3:
        n_3gram += 1
        
print(min_score)
print(max_score)
print(n_2gram)
print(n_3gram)

11.83010970221591
30.1391512738226
200
1800


In [15]:
train_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

train_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

In [16]:
from sklearn.svm import LinearSVC

from library.classification import SKLearnClassification
from library.evaluation import ConfusionMatrix

svm = LinearSVC()

model = SKLearnClassification(svm, "Support Vector Machine")
print(f"\n--- START ---")
model.train(train_vectors, train_labels, "Phemernr2")

print("Validation Set")
preds = model.predict(val_vectors)

conf_mat = ConfusionMatrix(
    labels=val_labels,
    predictions=preds,
    binary=False
)
conf_mat.evaluate()

print("--- END ---\n")


--- START ---
---> execution time : 0.12 seconds
Validation Set
Binary Class Evaluation

True Positive : 98
False Positive : 47
False Negative : 425
True Negative : 890

Class positive Evaluation
- Precision : 67.586 %
- Recall : 18.738 %
- F1 : 0.29341

Class negative Evaluation
- Precision : 67.681 %
- Recall : 94.984 %
- F1 : 0.79041

Combined Evaluation
- Accuracy : 67.671 %
- Precision : 67.633 %
- Recall : 56.861 %
- F1 : 0.61781
- Average Confidence : 132.88 %
Model, Combined,,,,positive,,,negative,,,
Anonymous, 67.671, 67.633, 56.861, 0.61781, 67.586, 18.738, 0.29341, 67.681, 94.984, 0.79041, 
--- END ---



In [17]:
from scipy.sparse import csr_matrix

coefs = model.model.coef_
if type(coefs) == csr_matrix:
    coefs.toarray().tolist()[0]
else:
    coefs.tolist()
coefs_and_features = list(zip(coefs[0], term_vector_base))

# Most predictive overall
# coefs_and_features = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)
coefs_and_features = sorted(coefs_and_features, key=lambda x: abs(x[0]), reverse=True)
print(f"Total tokens : {len(coefs_and_features)}")

Total tokens : 2000


In [18]:
coefs.shape

(4, 2001)

In [19]:
coefs_and_features

[(-1.6181117656488104, 'Martin Place Sydney'),
 (-1.5563629756381654, 'were on board'),
 (-1.4634366342435037, 'to U S'),
 (-1.4359348667927572, "isn't by Banksy"),
 (-1.3689238568029465, 'has been shot'),
 (-1.3627927350503262, 'die as martyrs'),
 (-1.3596612622221516, 'at Massey Hall'),
 (1.3567849553553817, 'do with Islam'),
 (-1.354653341704757, 'Chocolate Cafe in'),
 (-1.3545865696763195, 'Andreas Lubitz'),
 (-1.3288099914696943, "gunman's demands"),
 (-1.3265281684820645, 'Kunstmuseum Bern accepts'),
 (-1.3156578544051771, 'Canadian soldier killed'),
 (-1.3018692264787985, 'news conference on'),
 (-1.2998514093618216, 'has been named'),
 (-1.2878175641224305, 'running out of'),
 (-1.2846905321317883, 'University of Ottawa'),
 (-1.2617731222084996, 'number of hostages'),
 (1.2596500834894035, 'Paris offices of'),
 (-1.2543313540962442, 'French Interior Minister'),
 (-1.2543313055470449, "in Sydney's CBD"),
 (-1.2437104510698966, 'Swiss museum accepts'),
 (-1.243445522579404, 'are 

In [20]:
# print(max(f1s))
# f1s.index(max(f1s))

In [21]:
# import matplotlib.pyplot as plt

# x = [i*100 for i in range(len(f1s))]
# y = f1s

# plt.xlabel("Token/Bigram used")
# plt.ylabel("F1 Score")
# plt.plot(x, y)
# plt.show()

In [22]:
# import json

# best_tokens = [t[1] for t in coefs_and_features[:6100]]
# best_tokens[:10]
# 
# with open("../../data/processed/twitter15_best_bigrams.txt", "w") as f:
#     for token in best_tokens:
#         f.write(token + "\n")

In [23]:
coefs_and_features[:10]

[(-1.6181117656488104, 'Martin Place Sydney'),
 (-1.5563629756381654, 'were on board'),
 (-1.4634366342435037, 'to U S'),
 (-1.4359348667927572, "isn't by Banksy"),
 (-1.3689238568029465, 'has been shot'),
 (-1.3627927350503262, 'die as martyrs'),
 (-1.3596612622221516, 'at Massey Hall'),
 (1.3567849553553817, 'do with Islam'),
 (-1.354653341704757, 'Chocolate Cafe in'),
 (-1.3545865696763195, 'Andreas Lubitz')]

In [24]:
coefs_and_features[-10:]

[(0.0, 'flight #4U9525 with'),
 (0.0, '12 people have'),
 (0.0, 'everyone involved in'),
 (0.0, 'said it was'),
 (0.0, 'ever be #CharlieHebdo'),
 (0.0, 'robbery suspect #Ferguson'),
 (0.0, 'the satirical newspaper'),
 (0.0, 'Centre and Parliament'),
 (0.0, 'Kabaeva giving'),
 (0.0, 'wake up to')]

In [25]:
coefs_and_features[-10:]

[(0.0, 'flight #4U9525 with'),
 (0.0, '12 people have'),
 (0.0, 'everyone involved in'),
 (0.0, 'said it was'),
 (0.0, 'ever be #CharlieHebdo'),
 (0.0, 'robbery suspect #Ferguson'),
 (0.0, 'the satirical newspaper'),
 (0.0, 'Centre and Parliament'),
 (0.0, 'Kabaeva giving'),
 (0.0, 'wake up to')]

In [26]:
n_best = 750
best_tokens = []

# for cf in coefs_and_features[-n_best:]:
#     best_tokens.append(cf[1])

for cf in coefs_and_features[:n_best]:
    best_tokens.append(cf[1])
    
print(best_tokens[:10])
    
with open("../../data/processed/phemernr2-multiclass_best_terms.txt", "w") as f:
    for token in best_tokens:
        f.write(token + "\n")

['Martin Place Sydney', 'were on board', 'to U S', "isn't by Banksy", 'has been shot', 'die as martyrs', 'at Massey Hall', 'do with Islam', 'Chocolate Cafe in', 'Andreas Lubitz']
